<a href="https://colab.research.google.com/github/Gayeon6423/2023BigContest/blob/main/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Import module and Load Data

In [12]:
#  데이터 처리 모듈
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 행렬 등 통계 연산 모듈
import numpy as np
from scipy.stats import iqr
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

# 조합 찾는 반복문 메서드
from itertools import permutations
from itertools import combinations

# 데이터 시각화
import matplotlib.pyplot as plt
# from matplotlib.pyplot import
import matplotlib.font_manager as fm
import missingno as msno
import seaborn as sns

# pd.set_option('display.max_seq_items',5)

# 모델링 메서드
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 회귀 모델
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# # apt-get : 패키지 설치 명령어
# # -qq : 에러외의 메세지 숨기는 옵션키
# !apt-get -qq install fonts-nanum

# fe = fm.FontEntry(
#     fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
#     # ttf 파일이 저장되어 있는 경로
#     name='NanumGothic')
# # 이 폰트의 원하는 이름 설정
# fm.fontManager.ttflist.insert(0, fe)
# # Matplotlib에 폰트 추가
# plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})
# # 폰트 설정

In [14]:
class LoadGoogleDriveData():
  def __init__(self, data = None):
    self.data = data

  def loadData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
  def loadDataWithChunking(self, file_path: str, file_name_extension,
                           chunking_row_num: int, columnTF: bool, unicode: str) -> pd.DataFrame():
    chunkdata = pd.read_csv(os.path.join(file_path + file_name_extension),
                            chunksize = chunking_row_num,
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    self.data = list(chunkdata)
    self.data = pd.concat(self.data)

    return self.data

In [15]:
mountInstance = LoadGoogleDriveData()

In [56]:
KCD_MARKET_CONTEST_SEASON_2022 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/", file_name_extension = "KCD_MARKET_CONTEST_SEASON_2022.csv",
    unicode = "utf-8-sig", columnTF = False)

KCD_MARKET_CONTEST_SEASON_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/", file_name_extension = "KCD_MARKET_CONTEST_SEASON_2023.csv",
    unicode = "utf-8-sig", columnTF = False)

In [57]:
# 범주형 변수 처리
KCD_MARKET_CONTEST_SEASON_2022['INDUSTRY_CD'] = KCD_MARKET_CONTEST_SEASON_2022['INDUSTRY_CD'].str[1:].astype('int64')
KCD_MARKET_CONTEST_SEASON_2023['INDUSTRY_CD'] = KCD_MARKET_CONTEST_SEASON_2023['INDUSTRY_CD'].str[1:].astype('int64')

# 칼럼 순서 맞추기
col = KCD_MARKET_CONTEST_SEASON_2023['DATA_CRTR_YM']
KCD_MARKET_CONTEST_SEASON_2023 = KCD_MARKET_CONTEST_SEASON_2023.drop(columns = ['DATA_CRTR_YM'])
KCD_MARKET_CONTEST_SEASON_2023.insert(100, 'DATA_CRTR_YM', col)

## 1. 기본 데이터 모델링 성능 확인

In [ ]:
# 데이터 분할
X = KCD_MARKET_CONTEST_SEASON_2022.drop(columns = ['SLS_GRD'])
y = KCD_MARKET_CONTEST_SEASON_2022['SLS_GRD']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 1-1. 선형회귀 모델링

In [ ]:
# 선형회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 학습된 모델을 사용하여 예측
y_pred = model.predict(X_test)

# 회귀 모델 평가
mse = mean_squared_error(y_test, y_pred)  # 평균 제곱 오차
r2 = r2_score(y_test, y_pred)  # 결정 계수 (R-squared)

print("평균 제곱 오차 (MSE):", mse)
print("결정 계수 (R-squared):", r2)

평균 제곱 오차 (MSE): 1.2697404319731174
결정 계수 (R-squared): 0.03100133468744659


### 1-2. 의사결정나무 모델링

In [ ]:
# 모델 생성 및 학습
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# 학습된 모델을 사용하여 예측
y_pred = model.predict(X_test)

# 회귀 모델 평가
mse = mean_squared_error(y_test, y_pred)  # 평균 제곱 오차
r2 = r2_score(y_test, y_pred)  # 결정 계수 (R-squared)

print("평균 제곱 오차 (MSE):", mse)
print("결정 계수 (R-squared):", r2)

평균 제곱 오차 (MSE): 1.1730631508769487
결정 계수 (R-squared): 0.10478031658743969


### 1-3. 랜덤포레스트 모델링

In [ ]:
# 모델 생성 및 학습
model = RandomForestRegressor()
model.fit(X_train, y_train)

# 학습된 모델을 사용하여 예측
y_pred = model.predict(X_test)

# 회귀 모델 평가
mse = mean_squared_error(y_test, y_pred)  # 평균 제곱 오차
r2 = r2_score(y_test, y_pred)  # 결정 계수 (R-squared)

print("평균 제곱 오차 (MSE):", mse)
print("결정 계수 (R-squared):", r2)

평균 제곱 오차 (MSE): 1.1598800871595014
결정 계수 (R-squared): 0.11484093277738472


## 2. 1단계 모델링

### 2-1. 최근 데이터로 학습
- 22년 1월 ~ 22년 12월 데이터로 학습 => 23년 1월 매출등급 예측
- 22년 1월 ~ 23년 1월 데이터로 학습(23년 1월 예측값 사용) => 23년 2월 매출등급 예측

<img src = 'https://drive.google.com/uc?id=1qdrednUyVO60UYaX1s7XnW-XqwBC-jDC'>

In [63]:
# 23년 1월 매출등급 예측
KCD_MARKET_CONTEST_SEASON_2301 = KCD_MARKET_CONTEST_SEASON_2023[KCD_MARKET_CONTEST_SEASON_2023['DATA_CRTR_YM'] == 20231] # 23년1월 데이터

X_train = KCD_MARKET_CONTEST_SEASON_2022.drop(columns = ['SLS_GRD']) # 22년1월~12월 feature
y_train = KCD_MARKET_CONTEST_SEASON_2022['SLS_GRD'] # 22년1월~12월 target(매출등급)
KCD_MARKET_CONTEST_SEASON_2301 = KCD_MARKET_CONTEST_SEASON_2301.drop(columns = ['SLS_GRD_2301','SLS_GRD_2302']) # 23년1월 feature
X_test = KCD_MARKET_CONTEST_SEASON_2301.drop(columns = ['기준_년_코드','기준_월_코드'])

# 선형회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# # 학습된 모델을 사용하여 예측
y_pred_2301 = model.predict(X_test)
y_pred_2301 = pd.DataFrame({'SLS_GRD_2301_pred':y_pred_2301})
print('23년 1월 매출등급 예측')
y_pred_2301
# y_pred_2301.to_csv('y_pred_2301.csv',encoding='utf-8-sig',index=False)

23년 1월 매출등급 예측


,SLS_GRD_2301_pred
0,41.6072
1,41.2764
2,41.9002
3,41.2071
4,41.6842
...,...
24322,41.5313
24323,41.3890
24324,41.5243
24325,41.3825


In [ ]:
# 23년 2월 매출등급 예측
KCD_MARKET_CONTEST_SEASON_2302 = KCD_MARKET_CONTEST_SEASON_2023[KCD_MARKET_CONTEST_SEASON_2023['DATA_CRTR_YM'] == 20232] # 23년2월 데이터

KCD_MARKET_CONTEST_SEASON_2301 = KCD_MARKET_CONTEST_SEASON_2023[KCD_MARKET_CONTEST_SEASON_2023['DATA_CRTR_YM'] == 20231] # 23년1월 데이터
KCD_MARKET_CONTEST_SEASON_2201_2301 = pd.concat([KCD_MARKET_CONTEST_SEASON_2022, KCD_MARKET_CONTEST_SEASON_2301], axis=0, join='left') # 22년1월~23년1월 데이터
y_pred_2301 = pd.DataFrame(y_pred_2301)['SLS_GRD']
y_pred_2201_2301 = pd.concat([KCD_MARKET_CONTEST_SEASON_2301['SLS_GRD'],y_pred_2301],axis=0) # 22년1월~23년1월 매출등급

X_train = KCD_MARKET_CONTEST_SEASON_2201_2301.drop(columns = ['SLS_GRD']) # 22년1월~23년1월 feature
X_test = KCD_MARKET_CONTEST_SEASON_2302.drop(columns = ['SLS_GRD_2301','SLS_GRD_2302']) # 23년2월 feature
y_train = y_pred_2201_2301 # 22년1월~23년1월 target(매출등급)

# 선형회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 학습된 모델을 사용하여 예측
y_pred_2302 = model.predict(X_test)
print('23년 2월 매출등급 예측')
print(y_pred_2302)
# y_pred_2301.to_csv('y_pred_2301.csv',encoding='utf-8-sig',index=False)